In [19]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from scipy.ndimage import gaussian_filter


In [ ]:

# Correct path based on your folder structure
admin_regions = gpd.read_file("nepal_climate_data/nepal_admin_regions.gpkg")
print(admin_regions.head())  # Check if it loads correctly


In [20]:
#Load Nepal's GIS Data


# Load Nepal's administrative boundaries
nepal_admin = gpd.read_file("nepal_climate_data/nepal_admin_regions.gpkg")

# Load Nepal's rivers
nepal_rivers = gpd.read_file("nepal_climate_data/nepal_rivers.gpkg")

# Load Nepal's glaciers
nepal_glaciers = gpd.read_file("nepal_climate_data/nepal_glaciers.gpkg")

# Load local administrative units (Shapefile)
local_units = gpd.read_file("Shape_Data/local_unit.shp")


In [21]:
#Load Climate Raster Data (Temperature & Precipitation)

# Function to read raster files
def load_raster(file_path):
    with rasterio.open(file_path) as src:
        raster_data = src.read(1)  # Read first band
        transform = src.transform
    return raster_data, transform

# Load temperature and precipitation data
temp_2020, temp_transform = load_raster("nepal_climate_data/nepal_temperature_2020.tif")
temp_2050, _ = load_raster("nepal_climate_data/nepal_temperature_2050.tif")
precip_2020, precip_transform = load_raster("nepal_climate_data/nepal_precipitation_2020.tif")
precip_2050, _ = load_raster("nepal_climate_data/nepal_precipitation_2050.tif")


In [ ]:
#Plot Nepal's Administrative Boundaries with Rivers

fig, ax = plt.subplots(figsize=(10, 6))
nepal_admin.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)
nepal_rivers.plot(ax=ax, color="blue", linewidth=0.5, alpha=0.7)
plt.title("Nepal Administrative Boundaries & Rivers")
plt.show()


In [ ]:
#Visualize Temperature Data

fig, ax = plt.subplots(figsize=(10, 6))
show(temp_2020, transform=temp_transform, cmap="coolwarm", ax=ax)
nepal_admin.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)
plt.title("Temperature Distribution in Nepal (2020)")
plt.show()


In [ ]:
#Compare Temperature Changes (2020 vs. 2050)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# 2020 Temperature
show(temp_2020, transform=temp_transform, cmap="coolwarm", ax=axes[0])
nepal_admin.plot(ax=axes[0], edgecolor="black", facecolor="none", linewidth=1)
axes[0].set_title("Temperature 2020")

# 2050 Temperature
show(temp_2050, transform=temp_transform, cmap="coolwarm", ax=axes[1])
nepal_admin.plot(ax=axes[1], edgecolor="black", facecolor="none", linewidth=1)
axes[1].set_title("Temperature 2050")

plt.tight_layout()
plt.show()


In [ ]:
#Apply Gaussian Smoothing to Climate Data

smoothed_temp = gaussian_filter(temp_2020, sigma=2)
fig, ax = plt.subplots(figsize=(10, 6))
show(smoothed_temp, transform=temp_transform, cmap="coolwarm", ax=ax)
nepal_admin.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)
plt.title("Smoothed Temperature Data (2020)")
plt.show()


In [ ]:
#Plot Precipitation Data (2020 & 2050)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# 2020 Precipitation
show(precip_2020, transform=precip_transform, cmap="Blues", ax=axes[0])
nepal_admin.plot(ax=axes[0], edgecolor="black", facecolor="none", linewidth=1)
axes[0].set_title("Precipitation 2020")

# 2050 Precipitation
show(precip_2050, transform=precip_transform, cmap="Blues", ax=axes[1])
nepal_admin.plot(ax=axes[1], edgecolor="black", facecolor="none", linewidth=1)
axes[1].set_title("Precipitation 2050")

plt.tight_layout()
plt.show()


In [ ]:
#Compute Precipitation Change

precip_change = precip_2050 - precip_2020  # Change in precipitation
fig, ax = plt.subplots(figsize=(10, 6))
show(precip_change, transform=precip_transform, cmap="RdYlBu", ax=ax)
nepal_admin.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)
plt.title("Change in Precipitation (2020-2050)")
plt.colorbar(ax.images[0], label="Change in mm")
plt.show()


In [ ]:
#Compute Temperature Change

temp_change = temp_2050 - temp_2020  # Increase in temperature

fig, ax = plt.subplots(figsize=(10, 6))
show(temp_change, transform=temp_transform, cmap="coolwarm", ax=ax)
nepal_admin.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)
plt.title("Projected Temperature Change (2020-2050)")
plt.colorbar(ax.images[0], label="Temperature Change (°C)")
plt.show()


In [ ]:
#Compute Average Temperature Increase

avg_temp_increase = np.nanmean(temp_change)  # Ignore NaNs
print(f"Average Temperature Increase (2020-2050): {avg_temp_increase:.2f}°C")


In [ ]:
#Interactive Maps using folium

#Create a Base Map

import folium

# Define Nepal's center coordinates (approximate)
nepal_center = [28.3949, 84.1240]  
m = folium.Map(location=nepal_center, zoom_start=6)

# Add administrative boundaries
folium.GeoJson(nepal_admin, name="Nepal Admin Boundaries").add_to(m)

# Show the map
m


In [ ]:
#Add Temperature Change as a Heatmap

from folium.plugins import HeatMap

# Convert raster to list of (lat, lon, value)
temp_change_points = []
for row in range(temp_change.shape[0]):
    for col in range(temp_change.shape[1]):
        value = temp_change[row, col]
        if not np.isnan(value):  # Ignore NaNs
            lon, lat = temp_transform * (col, row)
            temp_change_points.append([lat, lon, value])

# Add heatmap layer
HeatMap(temp_change_points, radius=15, name="Temperature Change").add_to(m)

# Show map
m


In [32]:
#Statistical Analysis

#Compute Statistics
# Temperature statistics
temp_2020_mean = np.nanmean(temp_2020)
temp_2050_mean = np.nanmean(temp_2050)
temp_variance = np.nanvar(temp_2020)
temp_max = np.nanmax(temp_2050) - np.nanmax(temp_2020)

# Precipitation statistics
precip_2020_mean = np.nanmean(precip_2020)
precip_2050_mean = np.nanmean(precip_2050)
precip_variance = np.nanvar(precip_2020)
precip_max = np.nanmax(precip_2050) - np.nanmax(precip_2020)

print(f"Temperature 2020 Mean: {temp_2020_mean:.2f}°C")
print(f"Temperature 2050 Mean: {temp_2050_mean:.2f}°C")
print(f"Temperature Variance: {temp_variance:.2f}")
print(f"Maximum Temperature Increase: {temp_max:.2f}°C\n")

print(f"Precipitation 2020 Mean: {precip_2020_mean:.2f} mm")
print(f"Precipitation 2050 Mean: {precip_2050_mean:.2f} mm")
print(f"Precipitation Variance: {precip_variance:.2f}")
print(f"Maximum Precipitation Change: {precip_max:.2f} mm")


Temperature 2020 Mean: -7.58°C
Temperature 2050 Mean: -5.47°C
Temperature Variance: 53.22
Maximum Temperature Increase: 3.33°C

Precipitation 2020 Mean: 50.84 mm
Precipitation 2050 Mean: 48.29 mm
Precipitation Variance: 129.79
Maximum Precipitation Change: 32.14 mm
